## PCA e Classificação com Árvore de Decisão: Otimização da Dimensionalidade

## Introdução
Nesta análise, exploramos a aplicação da Análise de Componentes Principais (PCA) para reduzir a dimensionalidade do conjunto de dados HAR (Human Activity Recognition) e avaliamos seu impacto no desempenho de um classificador baseado em árvore de decisão.

Os principais objetivos são:
- Avaliar a acurácia do modelo sem redução de dimensionalidade.
- Aplicar o PCA e testar a influência da variação no número de componentes principais.
- Comparar o desempenho computacional e a acurácia nas bases de treino e teste.

In [1]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [4]:
filename_features = "features.txt"
filename_labels = "activity_labels.txt"

filename_subtrain = "subject_train.txt"
filename_xtrain = "X_train.txt"
filename_ytrain = "y_train.txt"

filename_subtest = "subject_test.txt"
ffilename_xtest = "X_test.txt"
filename_ytest = "y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], squeeze=True, sep="#")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], squeeze=True)
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], squeeze=True)
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

C:\Users\yanks\AppData\Local\Temp\ipykernel_29020\2403609656.py:12: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  features = pd.read_csv(filename_features, header=None, names=['nome_var'], squeeze=True, sep="#")
C:\Users\yanks\AppData\Local\Temp\ipykernel_29020\2403609656.py:15: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], squeeze=True)
C:\Users\yanks\AppData\Local\Temp\ipykernel_29020\2403609656.py:19: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], squeeze=True)


## Construção da Árvore de Decisão sem Redução de Dimensionalidade
Inicialmente, treinamos uma árvore de decisão utilizando todas as 561 variáveis disponíveis no dataset, sem nenhuma técnica de redução de dimensionalidade.

🔹 **Hiperparâmetro utilizado:**

`ccp_alpha = 0.001`: Aplicação de poda para evitar overfitting.

**Resultados Obtidos:**

- Acurácia na base de treino: $88%$
- Acurácia na base de teste: $87%$
- Tempo de processamento: $\sim3.19$ segundos

Esse desempenho reflete a capacidade do modelo de aprender padrões complexos nos dados, mas também sugere que a alta dimensionalidade pode estar influenciando a generalização.

In [6]:
%%time
# Medindo o tempo de processamento
# A partir deste ponto, será cronometrado o tempo necessário para executar o código abaixo.

# Criação do classificador de árvore de decisão com ccp_alpha=0.001
clf = DecisionTreeClassifier(ccp_alpha=0.001)

# Treinamento do classificador utilizando os dados de treinamento
clf.fit(X_train, y_train)

# Avaliação da acurácia do classificador nos dados de treinamento
print(f'Acurácia na base de treinamento: {clf.score(X_train, y_train)}')
# Avaliação da acurácia do classificador nos dados de teste
print(f'Acurácia na base de teste: {clf.score(X_test, y_test)}\n')

Acurácia na base de treinamento: 0.9757889009793254
Acurácia na base de teste: 0.8812351543942993

CPU times: total: 2.55 s
Wall time: 3.19 s


## Aplicação do PCA e Avaliação da Árvore de Decisão
O PCA foi aplicado ao conjunto de dados para reduzir a dimensionalidade, transformando as variáveis originais em componentes principais ortogonais. Para essa análise inicial, utilizamos apenas uma componente principal, avaliando sua influência na classificação.

#### Resultados com PCA ($k=1$ componente):

- Acurácia na base de treino: $49.97%$
- Acurácia na base de teste: $45.70%$
- Tempo de processamento: $\sim955$ ms

Observação: Com apenas uma componente principal, houve uma redução drástica no tempo de processamento, mas a acurácia caiu significativamente. Isso sugere que uma única componente não captura informações suficientes para a correta diferenciação das atividades.

In [7]:
%%time

# Aplica o PCA com 1 componente aos dados de treinamento
prcomp = PCA(n_components=1).fit(X_train)

# Transforma os dados de treinamento e teste utilizando as componentes principais encontradas pelo PCA
pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

# Imprime a forma dos dados de treinamento e teste após a transformação
print(f'Dimensões da base de treinamento: {pc_treino.shape}')
print(f'Dimensões da base de teste: {pc_teste.shape}')

# Inicializa um classificador de árvore de decisão com ccp_alpha=0.001 e treina-o com os dados de treinamento transformados
clf = DecisionTreeClassifier(ccp_alpha=0.001)
clf.fit(pc_treino, y_train)

# Calcula e imprime a acurácia do classificador nos dados de treinamento e teste
print(f'Acurácia na base de treinamento: {clf.score(pc_treino, y_train)}')
print(f'Acurácia na base de teste: {clf.score(pc_teste, y_test)}\n')

Dimensões da base de treinamento: (7352, 1)
Dimensões da base de teste: (2947, 1)
Acurácia na base de treinamento: 0.499727965179543
Acurácia na base de teste: 0.45707499151679676

CPU times: total: 312 ms
Wall time: 955 ms


## Teste com Diferentes Números de Componentes
Para entender o impacto do número de componentes principais ($k$), testamos os seguintes valores:

$$ k \in {1, 2, 5, 10, 50} $$


In [8]:
%%time

componentes = [1, 2, 5, 10, 50]

# Loop sobre os diferentes números de componentes
for n in componentes:
    # Executa o PCA com o número de componentes atual
    prcomp = PCA(n_components=n).fit(X_train)

    # Transforma os dados de treinamento e teste nos componentes principais
    pc_treino = prcomp.transform(X_train)
    pc_teste  = prcomp.transform(X_test)

    # Imprime as dimensões dos dados transformados
    print(f'Dimensões da base de treinamento: {pc_treino.shape}')
    print(f'Dimensões da base de teste: {pc_teste.shape}')

    # Cria e treina um classificador de árvore de decisão
    clf = DecisionTreeClassifier(ccp_alpha=0.001)
    clf.fit(pc_treino, y_train)

    # Avalia a acurácia na base de treinamento e teste
    print(f'Acurácia na base de treinamento: {clf.score(pc_treino, y_train)}')
    print(f'Acurácia na base de teste: {clf.score(pc_teste, y_test)}\n')

Dimensões da base de treinamento: (7352, 1)
Dimensões da base de teste: (2947, 1)
Acurácia na base de treinamento: 0.499727965179543
Acurácia na base de teste: 0.45707499151679676

Dimensões da base de treinamento: (7352, 2)
Dimensões da base de teste: (2947, 2)
Acurácia na base de treinamento: 0.6127584330794341
Acurácia na base de teste: 0.5846623685103495

Dimensões da base de treinamento: (7352, 5)
Dimensões da base de teste: (2947, 5)
Acurácia na base de treinamento: 0.8460282916213275
Acurácia na base de teste: 0.7885985748218527

Dimensões da base de treinamento: (7352, 10)
Dimensões da base de teste: (2947, 10)
Acurácia na base de treinamento: 0.8926822633297062
Acurácia na base de teste: 0.8238887003732609

Dimensões da base de treinamento: (7352, 50)
Dimensões da base de teste: (2947, 50)
Acurácia na base de treinamento: 0.9171653971708379
Acurácia na base de teste: 0.823549372242959

CPU times: total: 3.98 s
Wall time: 1.93 s


Observações:
- O aumento do número de componentes melhora progressivamente a acurácia, demonstrando que informações importantes estavam distribuídas entre múltiplas dimensões.
- Com 50 componentes principais, atingimos 82% de acurácia na base de teste, um desempenho próximo ao modelo original, porém com redução do tempo de processamento pela metade.
- A curva de aprendizado do modelo sugere que o uso de um número intermediário de componentes pode equilibrar eficiência computacional e performance preditiva.

##  Conclusão
Principais aprendizados deste estudo:
- Redução de Dimensionalidade: O PCA permitiu reduzir drasticamente o tempo de processamento, tornando o modelo mais eficiente.
- Impacto na Acurácia: Com poucas componentes, houve grande perda de performance. No entanto, com 50 componentes, o modelo se aproximou do desempenho original.
- Equilíbrio Ideal: Utilizar um número adequado de componentes pode preservar a maior parte da informação relevante, reduzindo a complexidade sem perda excessiva de precisão.